# API 設計

## Inplace 及 new object

一般而言，Julia 中設計 API 會有兩種版本，一種是直接修改原參數的版本，另一種是產生一個新的物件的版本。例如我們要做一個 `square` 來將輸入的矩陣內的元素全部平方，這時候會有 `square!` 直接修改矩陣內的元素，以及 `square` 來提供不修改原矩陣的版本。

In [1]:
function _square!(res::Matrix{T}, mat::Matrix{T}) where T
    for j = 1:size(mat,2)
        for i = 1:size(mat,1)
            res[i,j] = mat[i,j]^T(2)
        end
    end
end

_square! (generic function with 1 method)

我們會有一個 `_square!` 會將 `mat` 當中的元素取平方，然後寫入 `res`。這是在內部運作的函式，使用者並不會直接使用到這個函式，所以我們會給它的名字前面加一個 `_`，這是 Julia 中的慣例。這邊可以注意到的一個實作細節是，在平方的時候我們使用了 `T`，對 `2` 做轉型，`T(2)`。值得一提的是，型別在 Julia 中是一個物件，所以它也可以像物件一樣被操作。這麼做主要是為了型別穩定性，如此可以增加程式的效率，讓計算出來的值符合要寫入的矩陣元素型別。我們先將這樣的功能實作出來，將讀取資料及寫入資料的矩陣分開。我們接下來才是真正撰寫提供給使用者的介面。

In [2]:
function square!(mat::Matrix{T}) where T
    _square!(mat, mat)
    mat
end

square! (generic function with 1 method)

這邊我們提供 `square!(mat::Matrix{T})`，使用者輸入一個矩陣，並且會回傳處理完成的矩陣。實作上非常簡單易懂，讀取及寫入的矩陣為同一個矩陣，並且回傳 `mat` 物件。

In [3]:
function square(mat::Matrix{T}) where T
    res = similar(mat)
    _square!(res, mat)
    res
end

square (generic function with 1 method)

這邊我們提供 `square(mat::Matrix{T})`，使用者輸入一個矩陣，一個結果矩陣 `res` 會被建立，將結果寫入 `res`，並且回傳。這邊利用 `similar` 創造結果矩陣，當中包含未初始化的值，矩陣的元素型別及大小都與 `mat` 一致。

In [4]:
a = rand(1:5, 3, 4)

3×4 Array{Int64,2}:
 3  1  2  2
 5  5  1  4
 5  5  5  2

In [5]:
square(a)

3×4 Array{Int64,2}:
  9   1   4   4
 25  25   1  16
 25  25  25   4

In [6]:
a

3×4 Array{Int64,2}:
 3  1  2  2
 5  5  1  4
 5  5  5  2

In [7]:
square!(a)

3×4 Array{Int64,2}:
  9   1   4   4
 25  25   1  16
 25  25  25   4

In [8]:
a

3×4 Array{Int64,2}:
  9   1   4   4
 25  25   1  16
 25  25  25   4

## 直接回傳第一參數

我們可以在內部的實作中，直接回傳第一個參數，以方便後續的使用者介面實作。

In [9]:
function _square!(res::Matrix{T}, mat::Matrix{T}) where T
    for j = 1:size(mat,2)
        for i = 1:size(mat,1)
            res[i,j] = mat[i,j]^T(2)
        end
    end
    res
end

_square! (generic function with 1 method)

In [10]:
square(mat::Matrix{T}) where T = _square!(similar(mat), mat)
square!(mat::Matrix{T}) where T = _square!(mat, mat)

square! (generic function with 1 method)

以上我們可以注意到在 `_square!` 最後回傳了 `res`，如此一來，我們便可以在後續的實作當中直接回傳 `_square!`。介面變得非常簡單而且容易使用。

## 位置參數

Julia 的多重分派機制是依賴位置參數（positional arguments）的，完全不看關鍵字參數。多重分派是由位置參數的數量及型別來決定要呼叫什麼樣的方法。

In [11]:
foo(a) = println(a)
foo(a, b) = println((a, b))

foo (generic function with 2 methods)

In [12]:
foo(4)

4


In [13]:
foo(4, 5)

(4, 5)


位置參數可以有預設值，或是在文件中提到的可選參數（optional arguments）。可選參數就是在位置參數後加上預設值，位置參數沒有填上的部份會由預設值填補。

In [14]:
foo(a, b=1, c=2) = println((a, b, c))

foo (generic function with 3 methods)

In [15]:
foo(1)

(1, 1, 2)


In [16]:
foo(2, 3)

(2, 3, 2)


使用上會發現位置參數的預設值是依序填補的。當只有一個參數時，會預設使用 `b` 及 `c` 的預設值。當有兩個參數時，會預設使用 `c` 的預設值。前面定義的一個或兩個變數的版本不會被呼叫到，後續都優先呼叫三個參數版本的方法。

## 關鍵字參數

關鍵字參數一定需要提供預設值。關鍵字參數與位置參數之間需要以 `;` 分隔開來。

In [17]:
bar(a; b=1, c=2) = println((a, b, c))

bar (generic function with 1 method)

關鍵字參數在呼叫時可以完全省略。

In [18]:
bar(5)

(5, 1, 2)


關鍵字參數不在乎關鍵字的位置。

In [19]:
bar(5, b=7)

(5, 7, 2)


In [20]:
bar(5, c=2)

(5, 1, 2)


關鍵字參數可以置於位置參數的前或後，甚至穿插其中也可以。

In [21]:
bar(5, c=2, b=7)

(5, 7, 2)


In [22]:
bar(c=2, b=7, 5)

(5, 7, 2)


In [23]:
bar(b=7, 5, c=2)

(5, 7, 2)


## 混合使用

在一些語言當中，位置參數跟關鍵字參數的觀念是混在一起的，但在 Julia 有嚴格的區分。帶有預設值的位置參數不能以關鍵字參數的方式呼叫。

In [24]:
baz(a=1) = println(a)

baz (generic function with 2 methods)

In [25]:
baz(a=5)

ErrorException: function baz does not accept keyword arguments

在 API 的設計上，位置參數傾向用於呼叫一個功能所必須的參數上，而關鍵字參數則相反。關鍵字參數會大量使用在繪圖的細節調整上，像是圖片大小、字體、點的大小等等。